# Cách tinh chỉnh LLM với LoRA Adapters sử dụng Hugging Face TRL

Trong Notebook này bạn sẽ được học cách tinh chỉnh hiệu quả các mô hình ngôn ngữ lớn sử dụng LoRA (Low-Rank Adaptation) adapters. LoRA là một kỹ thuật tinh chỉnh tham số hiệu quả:
- Đóng băng các trọng số mô hình đã huấn luyện trước
- Thêm các ma trận phân rã hạng nhỏ có thể huấn luyện vào các lớp attention  
- Thường giảm khoảng 90% tham số có thể huấn luyện
- Duy trì hiệu suất mô hình trong khi sử dụng bộ nhớ hiệu quả

Chúng ta sẽ tìm hiểu:  
1. Cài đặt môi trường phát triển và cấu hình LoRA
2. Tạo và chuẩn bị dữ liệu để huấn luyện adapter 
3. Tinh chỉnh sử dụng `trl` và `SFTTrainer` với LoRA adapters
4. Kiểm tra mô hình và gộp adapters (tùy chọn)

## 1. Cài đặt môi trường phát triển

Bước đầu tiên là cài đặt các thư viện Hugging Face và PyTorch, bao gồm `trl, transformers và datasets`. Nếu bạn chưa nghe nói về `trl`, đừng lo lắng. Đó là một thư viện mới trên nền tảng transformers và datasets, giúp việc tinh chỉnh các mô hình ngôn ngữ lớn (LLM) trở nên dễ dàng hơn.

In [ ]:
# Cài đặt các yêu cầu trong Google Colab
# !pip install transformers datasets trl huggingface_hub

# Xác thực với Hugging Face
from huggingface_hub import login

login()

# để thuận tiện bạn có thể tạo một biến môi trường chứa token hub của bạn là HF_TOKEN

## 2. Tải dữ liệu 

In [13]:
# Tải một tập dữ liệu mẫu
from datasets import load_dataset

# TODO: xác định dataset và config của bạn sử dụng các tham số path và name
dataset = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

## 3. Tinh chỉnh LLM sử dụng `trl` và `SFTTrainer` với LoRA

[SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) từ `trl` cung cấp tích hợp với LoRA adapters thông qua thư viện [PEFT](https://huggingface.co/docs/peft/en/index). Những lợi thế chính của cài đặt này bao gồm:

1. **Hiệu quả bộ nhớ**:
   - Chỉ các tham số adapter được lưu trữ trong bộ nhớ GPU 
   - Trọng số mô hình cơ sở vẫn đóng băng và có thể được tải với độ chính xác thấp hơn
   - Cho phép tinh chỉnh các mô hình lớn trên GPU tiêu dùng

2. **Tính năng huấn luyện**:
   - Tích hợp PEFT/LoRA sẵn có với cài đặt tối thiểu
   - Hỗ trợ QLoRA (LoRA lượng tử hóa) cho hiệu quả bộ nhớ tốt hơn

3. **Quản lý Adapter**: 
   - Lưu trọng số adapter trong quá trình checkpoint
   - Tính năng gộp adapters trở lại mô hình cơ sở

Chúng ta sẽ sử dụng LoRA trong ví dụ của mình, **kết hợp LoRA với lượng tử hóa 4-bit** để giảm thêm việc sử dụng bộ nhớ mà không ảnh hưởng đến hiệu suất. Cài đặt chỉ yêu cầu một vài bước cấu hình:
1. Xác định cấu hình LoRA (rank, alpha, dropout)
2. Tạo SFTTrainer với cấu hình PEFT 
3. Huấn luyện và lưu trọng số adapter

In [ ]:
# Các thư viện cần thiết 
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Tải mô hình và tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Thiết lập định dạng chat
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Đặt tên cho bản tinh chỉnh để lưu &/ tải lên
finetune_name = "SmolLM2-FT-LoRA-Adapter"
finetune_tags = ["smol-course", "module_3"]

`SFTTrainer` hỗ trợ tích hợp sẵn với `peft`, điều này giúp tinh chỉnh hiệu quả các LLM dễ dàng hơn bằng cách sử dụng, ví dụ như LoRA. Chúng ta chỉ cần tạo `LoraConfig` và cung cấp nó cho trainer.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập: Xác định tham số LoRA cho tinh chỉnh</h2>
    <p>Lấy một bộ dữ liệu từ Hugging Face hub và tinh chỉnh một mô hình trên nó.</p>
    <p><b>Mức độ khó</b></p> 
    <p>🐢 Sử dụng các tham số chung cho một bản tinh chỉnh tùy ý</p>
    <p>🐕 Điều chỉnh các tham số và so sánh trong weights & biases.</p>
    <p>🦁 Điều chỉnh các tham số và cho thấy sự thay đổi trong kết quả suy luận.</p>
</div>

In [ ]:
from peft import LoraConfig

# TODO: Điều chỉnh cấu hình tham số LoRA
# r: chiều rank cho ma trận cập nhật LoRA (nhỏ hơn = nén nhiều hơn)
rank_dimension = 4
# lora_alpha: hệ số tỷ lệ cho các lớp LoRA (cao hơn = điều chỉnh mạnh hơn) 
lora_alpha = 8
# lora_dropout: xác suất dropout cho các lớp LoRA (giúp tránh overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Chiều rank - thường từ 4-32
    lora_alpha=lora_alpha,  # Hệ số tỷ lệ LoRA - thường gấp 2 lần rank
    lora_dropout=lora_dropout,  # Xác suất dropout cho các lớp LoRA 
    bias="none",  # Loại bias cho LoRA. các bias tương ứng sẽ được cập nhật trong quá trình huấn luyện.
    target_modules="all-linear",  # Những module nào áp dụng LoRA
    task_type="CAUSAL_LM",  # Loại tác vụ cho kiến trúc mô hình
)

Trước khi bắt đầu huấn luyện, chúng ta cần xác định các `siêu tham số (TrainingArguments)` mà chúng ta muốn sử dụng.

In [ ]:
# Cấu hình huấn luyện  
# Siêu tham số dựa trên khuyến nghị từ bài báo QLoRA 
args = SFTConfig(
    # Cài đặt đầu ra
    output_dir=finetune_name,  # Thư mục để lưu checkpoint mô hình
    # Thời gian huấn luyện
    num_train_epochs=1,  # Số epoch huấn luyện
    # Cài đặt kích thước batch 
    per_device_train_batch_size=2,  # Kích thước batch cho mỗi GPU
    gradient_accumulation_steps=2,  # Tích lũy gradient cho batch hiệu quả lớn hơn
    # Tối ưu bộ nhớ
    gradient_checkpointing=True,  # Đánh đổi tính toán để tiết kiệm bộ nhớ
    # Cài đặt optimizer
    optim="adamw_torch_fused",  # Sử dụng AdamW fusion cho hiệu quả
    learning_rate=2e-4,  # Tốc độ học (từ bài báo QLoRA)
    max_grad_norm=0.3,  # Ngưỡng cắt gradient
    # Lịch trình học
    warmup_ratio=0.03,  # Phần bước cho warmup
    lr_scheduler_type="constant",  # Giữ tốc độ học không đổi sau warmup
    # Ghi log và lưu
    logging_steps=10,  # Ghi metrics mỗi N bước
    save_strategy="epoch",  # Lưu checkpoint mỗi epoch
    # Cài đặt độ chính xác
    bf16=True,  # Sử dụng độ chính xác bfloat16
    # Cài đặt tích hợp
    push_to_hub=False,  # Không đẩy lên HuggingFace Hub
    report_to=None,  # Tắt ghi log bên ngoài
)

Bây giờ chúng ta đã có mọi thứ cần thiết để tạo `SFTTrainer` và bắt đầu huấn luyện mô hình của mình.

In [ ]:
max_seq_length = 1512  # độ dài chuỗi tối đa cho mô hình và đóng gói (packing) bộ dữ liệu

# Tạo SFTTrainer với cấu hình LoRA
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,  # Cấu hình LoRA
    max_seq_length=max_seq_length,  # Độ dài chuỗi tối đa 
    tokenizer=tokenizer,
    packing=True,  # Bật đóng gói đầu vào cho hiệu quả 
    dataset_kwargs={
        "add_special_tokens": False,  # Token đặc biệt được xử lý bởi template
        "append_concat_token": False,  # Không cần thêm separator 
    },
)

Bắt đầu huấn luyện mô hình bằng cách gọi phương thức `train()` trên `Trainer` của chúng ta. Việc này sẽ bắt đầu vòng lặp huấn luyện và huấn luyện mô hình của chúng ta trong `3 epochs`. Vì chúng ta đang sử dụng phương pháp PEFT, chúng ta sẽ chỉ lưu phần trọng số của adapter đã điều chỉnh và không lưu toàn bộ mô hình.

In [ ]:
# bắt đầu huấn luyện, mô hình sẽ tự động được lưu lên hub và thư mục đầu ra
trainer.train()

# lưu mô hình
trainer.save_model()

  0%|          | 0/72 [00:00<?, ?it/s]

TrainOutput(global_step=72, training_loss=1.6402628521124523, metrics={'train_runtime': 195.2398, 'train_samples_per_second': 1.485, 'train_steps_per_second': 0.369, 'total_flos': 282267289092096.0, 'train_loss': 1.6402628521124523, 'epoch': 0.993103448275862})

Việc huấn luyện với Flash Attention cho 3 epoch với một dataset 15k mẫu mất `4:14:36` trên một cụm máy `g5.2xlarge` của AWS. Cụm máy này có giá `1.21$/h` và tổng chi phí của lần huấn luyện này chỉ tốn khoảng `5.3$`.

**Ghi chú: bạn hoàn toàn có thể sử dụng GPU của Kaggle hoặc Google Colab để huấn luyện**

### Gộp LoRA Adapter vào Mô hình Gốc

Khi sử dụng LoRA, chúng ta chỉ huấn luyện trọng số adapter trong khi giữ nguyên mô hình cơ sở. Trong quá trình huấn luyện, chúng ta chỉ lưu những trọng số adapter nhẹ này (~2-10MB) thay vì một bản sao mô hình đầy đủ. Tuy nhiên, để triển khai, bạn có thể muốn gộp các adapter trở lại mô hình cơ sở để:

1. **Đơn giản hóa triển khai**: Một file mô hình thay vì mô hình cơ sở + adapters
2. **Tốc độ suy luận**: Không có chi phí tính toán adapter phụ thêm
3. **Tương thích Framework**: Tương thích tốt hơn với các framework phục vụ

In [ ]:
from peft import AutoPeftModelForCausalLM


# Tải mô hình PEFT trên CPU  
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Gộp LoRA và mô hình cơ sở và lưu
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. Kiểm tra Mô hình

Sau khi huấn luyện hoàn tất, chúng ta muốn kiểm tra mô hình của mình. Chúng ta sẽ tải các mẫu khác nhau từ dataset gốc và đánh giá mô hình trên những mẫu đó, sử dụng một vòng lặp đơn giản và độ chính xác làm điểm số đánh giá.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bài tập Bonus: Tải LoRA Adapter</h2>
    <p>Sử dụng những gì bạn đã học được từ notebook ví dụ để tải adapter LoRA đã huấn luyện của bạn cho suy luận.</p>
</div>

In [30]:
# giải phóng bộ nhớ một lần nữa
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM 
from transformers import AutoTokenizer, pipeline

# Tải Mô hình với PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(finetune_name)
model = AutoPeftModelForCausalLM.from_pretrained(
    finetune_name, device_map="auto", torch_dtype=torch.float16
)
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=device
)

Hãy thử một số chỉ định mẫu và xem mô hình hoạt động như thế nào.

In [34]:
prompts = [
    "Thủ đô của Việt Nam thành phố nào? Giải thích tại sao là như vậy và liệu nó có khác trong quá khứ không?",
    "Viết một hàm Python để tính giai thừa của một số.",
    "Một khu vườn hình chữ nhật có chiều dài 25 mét và chiều rộng 15 mét. Nếu bạn muốn xây một hàng rào xung quanh toàn bộ khu vườn, bạn sẽ cần bao nhiêu mét hàng rào?",
    "Sự khác biệt giữa trái cây và rau củ là gì? Đưa ra ví dụ cho mỗi loại.",
]


def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
    )
    return outputs[0]["generated_text"][len(prompt):].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-" * 50)